In [1]:
# Notebook para generar el dataset de parcelas

In [2]:
import os
import pandas as pd

import geopandas as gpd
from fiona.drvsupport import supported_drivers
supported_drivers['KML'] = 'rw'

In [3]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
DATA_PATH = ROOT_PATH + '/data'

In [4]:
# Carga de todos los ficheros del catastro, en formato KML.

def LoadCatastroKML(s_file, s_path):
    file_polygons = gpd.read_file(s_path + '/' + s_file + '.kml', driver='KML')
    df_parc = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude'])
    
    for index, row in file_polygons.iterrows():
        l_lat = []
        l_long = []
        if len(row['Name']) > 0:
            s_sub = str(row['Name'])
        else:
            s_sub = 'z'
        
        for point in list(row['geometry'].exterior.coords): 
            l_lat.append(point[1])
            l_long.append(point[0])

        row_new = {'CatReferencia': s_file, 'CatRecinto': s_sub, 'Latitude': l_lat, 'Longitude': l_long}
        df_new = pd.DataFrame([row_new])

        df_parc = pd.concat([df_parc, df_new], axis=0, ignore_index=True)
        
        df_parc.fillna('', inplace=True)
        df_parc.loc[df_parc['CatRecinto'] == '0', 'CatRecinto'] = '-'

    return df_parc


In [5]:
# Función para cargar todos los ficheros de parcelas del Catastro (desde una carpeta)

def LoadCatastro(layer):
    
    # Obtengo los ficheros KML a cargar
    files_list = []
    s_path = DATA_PATH + '/layer-' + layer + '/KML'
    for file in os.listdir(s_path):
        if file.endswith(".kml"):
            files_list.append(file.replace('.kml', ''))

    df_parcels = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude'])

    # Leo la información del fichero KML y la añado al dataframe
    for file in files_list:
        df_new = LoadCatastroKML(file, s_path)
        df_parcels = pd.concat([df_parcels, df_new], axis=0, ignore_index=True)

    return df_parcels


In [6]:
# Carga información de las parcelas

def LoadParcelsExcel(layer):
    
    # Leemos el fichero de parcelas
    df_info = pd.read_excel (DATA_PATH + '/layer-' + layer + '/parcels.xls', sheet_name='parcels')
    df_info['CodigoCultivo'].fillna('', inplace=True)
    df_info['CodigoPropietario'].fillna('', inplace=True)
    df_info.loc[df_info['CatRecinto'] == 0, 'CatRecinto'] = '-'

    # Cargamos los ficheros de configuración
    df_c = pd.read_csv(CONFIG_PATH + '/cultivos.cfg', sep=',', encoding='UTF-8', na_filter= False)
    df_p = pd.read_csv(CONFIG_PATH + '/propietarios.cfg', sep=',', encoding='UTF-8', na_filter= False)

    # Completamos la información de las parcelas
    df_info = pd.merge(df_info, df_c, on=['CodigoCultivo'], how='left')
    df_info = pd.merge(df_info, df_p, on=['CodigoPropietario'], how='left')
    df_info['Origen'] = 'layer-' + layer
   
    # Y adaptamos el dataset al formato requerido
    df_info.fillna('', inplace=True)
    df_info.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)

    return df_info


In [7]:
# Función que guarda el dataset de las parcelas para ser visualizado en Google Maps

def SaveDataset(layer, df):

    s_file = 'dataset-parcels.pkl'
    s_path = DATA_PATH + '/layer-' + layer
    df.to_pickle(s_path + '/' + s_file)
    

In [8]:
# Función que fusiona la información del Catastro con la propia de la empresa 
# para generar el dataset que se visualizará postariormente

def GenerateDatasetParcels(layer):
    s_path = DATA_PATH + '/layer-' + layer
    
    if (os.path.isdir(s_path)) & (os.path.isfile(s_path + '/parcels.xls')):
        # Cargamos las parcelas del Catastro
        df_parcelas = LoadCatastro(layer)

        # Cargamos la información complementaria de las parcelas
        df_infoParcelas = LoadParcelsExcel(layer)   

        # Completamos la información de las parcelas
        df_compl = pd.merge(df_parcelas, df_infoParcelas, on=['CatReferencia', 'CatRecinto'], how='left')
        df_compl.fillna('', inplace=True)

        # Volcamos a disco el dataset
        SaveDataset(layer, df_compl)
        

In [9]:
# Función que genera los datasets individuales de las parcelas de todas las capas

def GenerateAllDatasetsParcels():
    for root, dirs, files in os.walk(DATA_PATH, topdown=False):
        for file_name in files:
            s_folder = os.path.basename(root)
            if (file_name == 'parcels.xls') & (s_folder.split('-')[0] == 'layer'):
                if s_folder.split('-')[1] is not None:
                    GenerateDatasetParcels(s_folder.split('-')[1])
 

## Ejecución

In [10]:
# Generación de todos los datasets de parcelas
GenerateAllDatasetsParcels()

# Generación de un dataset de parcelas
#GenerateDatasetParcels(layer='parcelas')